# 1. ollama 서버 상태 확인
#### 서버 실행되지 않았을 때, ConnectionError 방지

In [1]:
import requests

# 서버 연결 체크 함수 
def check_ollama_status(base_url="http://localhost:11434"):
    try:
        response = requests.get(base_url)
        return response.status_code == 200
    except requests.exceptions.ConnectionError:
        return False

# 서버가 켜져 있는지 확인
if not check_ollama_status():
    print("[ERROR] Ollama 서버가 실행 중이지 않습니다.")
    print("터미널에서 'ollama serve'를 실행하거나, 설치 여부를 확인하세요.")


# 2. Few-Shot Prompting Projects
## 2-1. 코드 리뷰 및 컨벤션 체크 (Code Reviewer)
#### 외부 서버로 코드를 보낼 수 없는 보안 환경에서 코드 리뷰 및 컨벤션 체크 (Code Reviewer)할 수 있는 로컬 LLM 프로젝트
#### 사내의 특정 코딩 스타일 가이드나 보안 규칙을 로컬 LLM에게 가르치는 프로젝트
#### 2-1-1. 규칙의 구체화 (Prefix): Prefix에 사내 위키(Wiki) 등에 정리된 코딩 스타일 가이드라인을 요약해서 입력
#### 2-1-2. Task에 맞는 모델 선택
###### - 성능 중심: Llama-3-8B
###### - 코드 특화: CodeLlama-7B, DeepSeek-Coder-6.7B
###### - 저사양 환경: Phi-3-mini
#### 2-1-3. CI/CD 연동: 이 프로젝트를 Git의 Pre-commit hook으로 등록해, 커밋 시 로컬 모델이 자동으로 코드 검사 후, 통과 못 하면 커밋 막는 기능 구현 해볼까?

In [ ]:
import requests
from langchain_community.llms import Ollama
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

# 1. 로컬 LLM 설정 (코드 리뷰에 특화된 모델 검색 e.g CodeLlama, DeepSeek-Coder)
llm = Ollama(model="codellama", temperature=0)

# 2. 사내 컨벤션 및 보안 규칙 Few-shot 예시 정의
examples = [
    {
        "bad_code": "var user_name = 'admin';",
        "review": """- [컨벤션] 'var' 대신 'const' 또는 'let'을 사용하세요.
- [명명법] 변수명은 camelCase(userName)를 권장합니다.
- [보안] 하드코딩된 계정 정보가 있는지 확인이 필요합니다."""
    },
    {
        "bad_code": "if(a == 10) {{ return true; }}",  # # [20260112] 예시 데이터의 코드 내 중괄호를 {{ }}로 변경
        "review": """- [컨벤션] 동등 연산자(==) 대신 일치 연산자(===)를 사용하여 타입 안전성을 확보하세요.
- [가독성] 변수명 'a'는 의미를 알기 어렵습니다. 역할을 알 수 있는 이름으로 수정하세요."""
    },
    {
        "bad_code": "const query = 'SELECT * FROM users WHERE id = ' + id;",
        "review": """- [보안] SQL Injection 위험이 있습니다. Parameterized Query 또는 ORM을 사용하세요.
- [보안] 유저 전체 정보(*) 대신 필요한 컬럼만 명시하세요."""
    }
]

# 3. 예시 포맷 정의
example_prompt = PromptTemplate(
    input_variables=["bad_code", "review"],
    template="[대상 코드]\n{bad_code}\n\n[리뷰 결과]\n{review}"
)

# 4. 전체 Few-shot 프롬프트 구성
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="""당신은 사내 코드 리뷰어입니다. 다음 규칙을 준수하여 입력된 코드를 검토하세요:
1. 변수명은 camelCase를 사용한다.
2. 'var' 사용을 금지한다.
3. 보안 취약점(SQL Injection, 하드코딩 등)을 최우선으로 지적한다.
4. 답변은 간결하게 리스트 형식으로 출력한다.""",
    suffix="\n[대상 코드]\n{input}\n\n[리뷰 결과]\n",
    input_variables=["input"],
    example_separator="\n\n---\n\n"
)

# 5. 코드 리뷰 실행
target_code = """
function save_data(input_val) {
    var query = "INSERT INTO logs VALUES ('" + input_val + "')";
    db.execute(query);
}
"""

final_prompt = few_shot_prompt.format(input=target_code)

print("로컬 LLM 코드 리뷰 GO!GO!GO!")
print("-" * 30)

try:
    response = llm.invoke(final_prompt)
    print(response)
except Exception as e:
    print(f"오류 발생: {e}\nOllama 서버가 실행 중인지 확인하세요.")

## 2-2. 정형 데이터 추출기 (Structured Data Extractor)
#### 2-2-1. Temperature 설정 (temperature=0): 정확한 답변 위해
#### 2-2-2. 품질의 예시 (Examples): Few-shot의 핵심은 "모델이 패턴을 인지하게 하는 것"
#### 2-2-3. Prefix와 Suffix의 역할
###### - Prefix: "반드시 JSON 포맷만 출력하세요"라는 제약 사항을 걸어 모델이 "네, 알겠습니다" 같은 불필요한 서두를 떼지 않게 함
###### - Suffix: 사용자의 실제 입력값(input_mail)을 배치하고, 마지막에 추출 결과(JSON):라는 문구를 두어 모델이 바로 다음 토큰으로 {를 생성하도록 유도
#### 2-2-4. 일부 데이터 누락된 예시 포함: (아는 척 못 하게) Hallucination 방지

In [ ]:
import json
from langchain_community.llms import Ollama
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

# 1. 로컬 LLM 설정 (예: llama3, mistral, 혹은 한국어 특화 모델)
llm = Ollama(model="llama3", temperature=0) # TODO 추출 작업은 정밀도 중요 temperature=0 설정

# 2. Few-shot을 위한 예시 데이터 (Examples) 정의
# 모델에게 어떤 비정형 텍스트에서 어떤 JSON 결과가 나와야 하는지 학습 예시 [20260112] 예시 데이터의 JSON 중괄호를 {{ }}로 변경
examples = [
    {
        "mail_text": "안녕하세요, 구글 코리아의 데이터 분석가 김철수입니다. 2023년 5월 10일에 입사하여 업무 보고 드립니다.",
        "answer": """{{"name": "김철수", "company": "구글 코리아", "role": "데이터 분석가", "date": "2023-05-10", "summary": "입사 후, 업무 보고"}}"""
    },
    {
        "mail_text": "네이버 마케팅팀 팀장 박영희입니다. 오는 2024년 1월 15일 미팅 일정 확인 부탁드립니다.",
        "answer": """{{"name": "박영희", "company": "네이버", "role": "마케팅팀 팀장", "date": "2024-01-15", "summary": "미팅 일정 확인"}}"""
    },
    {
        "mail_text": "삼성전자 폰팔이 이재용입니다. 프로젝트는 2023년 12월 25일에 종료될 예정입니다.",
        "answer": """{{"name": "이재용", "company": "삼성전자", "role": "폰팔이", "date": "2023-12-25", "summary": "프로젝트 종료 예정"}}"""
    },
    {
        "mail_text": "현대 중고차 딜러 정몽구입니다. 차보러 언제 오시나요?",  # # TODO [중요] 누락된 데이터(예: 날짜가 없는 경우)에 "정보 없음" 혹은 **null** 출력하는 예시 포함해 Hallucination 방지
        "answer": """{{"name": "정몽구", "company": "현대 중고차", "role": "중고차 딜러", "date": "정보 없음", "summary": "방문 일정"}}"""
    },
]

# 3. 예시를 보여줄 포맷(Template) 정의
example_formatter_template = """
메일 내용: {mail_text}
추출 결과(JSON): {answer}
"""
example_prompt = PromptTemplate(
    input_variables=["mail_text", "answer"],
    template=example_formatter_template,
)

# 4. Few-shot 프롬프트 구성
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    # 지시사항 부분에 예시 형식을 적을 때도 {{ }} 사용
    prefix="사내 메일에서 정보를 추출하여 {{\"name\": \"...\", \"company\": \"...\"}} 형식의 JSON으로 답하세요.",
    # prefix="다음 사내 메일 텍스트에서 이름, 회사, 역할, 날짜를 추출하여 JSON 형식으로 답변하세요. 반드시 JSON 포맷만 출력하세요.",
    suffix="메일 내용: {input}\n추출 결과(JSON):",
    input_variables=["input"],
    example_separator="\n\n",
)

# 5. 실행
input_mail = """
안녕하세요.

경영지원팀 이세돌 입니다.

엑셀 파일도 작성 후 회신 부탁 드립니다.

감사합니다.
"""
final_prompt = few_shot_prompt.format(input=input_mail)

print("--- [생성된 프롬프트] ---")
print(final_prompt)

print("\n--- [LLM 추출 결과] ---")
response = llm.invoke(final_prompt)
print(response)

# 6. JSON 파싱
try:
    data = json.loads(response)
    print("\n[파싱 성공]")
    print(f"이름: {data['name']}, 주요내용: {data['summary']}, 날짜: {data['date']}, 직책: {data['role']}, 회사: {data['company']}")
except Exception as e:
    print("\n[파싱 실패]", e)

## 2-3. 로컬 LLM 기반 데이터 분석가(Local Data Scientist) 구축
#### 민감한 로컬 데이터를 외부로 유출하지 않고도 정교한 분석 보고서를 작성
#### 2-3-1. 프로젝트 설계 구조
###### - 모델: Llama-3-8B(일반적), Mistral(분석 특화)
###### - 데이터 전략: 원본 데이터 전체를 프롬프트에 넣기보다는, **데이터의 샘플(Top 5 rows)**과 통계 요약 정보를 추출하여 프롬프트에 제공
###### - Few-shot 예시: '데이터 요약 → 가설 설정 → 분석 결과 → 비즈니스 제언'으로 이어지는 보고서 형식 학습



In [ ]:
import json
from langchain_community.llms import Ollama   # # from langchain_ollama import OllamaLLM  # # 차후 최신 OllamaLLM 로 변경 필요
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

# 1. 로컬 LLM 설정
llm = Ollama(model="mistral", temperature=0) #
# llm = OllamaLLM(model="mistral", temperature=0)  # # 차후 최신 OllamaLLM 로 변경 필요

# 2. 데이터 분석 Few-shot 예시 정의 (데이터 요약 -> 가설 -> 결과 -> 제언)
examples = [
    {
        "raw_data": "A쇼핑몰 8월 매출: 전월 대비 여성 의류 30% 증가, 남성 의류 5% 감소, 장바구니 포기율 40%",
        "analysis_report": """
1. 데이터 요약: 8월 매출은 여성 의류가 성장을 주도했으나 남성 부문은 정체됨.
2. 가설 설정: 시즌 종료 세일이 여성 고객에게만 더 강력한 소구력을 가졌을 것이다.
3. 분석 결과: 데이터 확인 결과 여성 고객의 재방문율이 남성보다 2.5배 높음.
4. 비즈니스 제언: 9월 신상품 런칭 시 남성 고객 전용 '첫 구매 할인 쿠폰'을 발행하여 이탈을 방지해야 함.
"""
    },
    {
        "raw_data": "클라우드 서버 가용성: 장애 시간 2시간 발생, CPU 점유율 90% 지속, 특정 API 호출 급증",
        "analysis_report": """
1. 데이터 요약: 특정 시간대 서버 부하로 인한 가용성 저하 발생.
2. 가설 설정: 신규 런칭한 모바일 앱의 푸시 알림이 API 서버에 동시 접속 부하를 유도했을 것이다.
3. 분석 결과: 푸시 발송 시점과 API 호출 급증 시점이 99% 일치함이 확인됨.
4. 비즈니스 제언: 푸시 알림 발송 시 '배치(Batch)' 방식으로 시간차 분산 발송 로직을 도입해야 함.
"""
    }
]

# 3. 예시 포맷 정의
example_prompt = PromptTemplate(
    input_variables=["raw_data", "analysis_report"],
    template="[입력 데이터]\n{raw_data}\n\n[분석 결과 보고서]\n{analysis_report}"
)

# 4. 전체 Few-shot 프롬프트 구성
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="""당신은 데이터 과학자입니다. 제공된 데이터를 분석하여 보고서를 작성하세요.
반드시 아래 4단계 구조를 엄격히 지켜 한국어로만 작성하세요:
1. 데이터 요약
2. 가설 설정
3. 분석 결과
4. 비즈니스 제언""",
    suffix="\n[입력 데이터]\n{input}\n\n[분석 결과 보고서]\n1. 데이터 요약:",
    input_variables=["input"],
    example_separator="\n\n---\n\n"
)

# 5. 실제 데이터 분석 실행
# 실 서비스 시, pandas.describe() 등의 통계값 추가하는 방향???
target_data = "배달 앱 10월 로그: 야간 시간대(22시~02시) 주문 15% 하락, 배달비 평균 2,000원 상승, 라이더 수 10% 감소"

final_prompt = few_shot_prompt.format(input=target_data)

print("로컬 데이터 분석가가 보고서를 생성 중...")
print("-" * 30)

try:
    # 6. 실행
    response = llm.invoke(final_prompt)
    # Suffix에서 시작한 문구를 합쳐서 출력
    print("1. 데이터 요약:" + response)
except Exception as e:
    print(f"연결 오류: {e}. Ollama 서버가 켜져 있는지 확인하세요.")